In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

In [2]:
import aliases

# inv_trainer = aliases.load_trainer_from_alias("openai_msmarco__msl128__100epoch")
corr_experiment, corr_trainer = aliases.load_experiment_and_trainer_from_alias("openai_msmarco__msl128__100epoch__correct")
inv_trainer = corr_trainer.inversion_trainer
corr_trainer.precompute_hypotheses()
corr_trainer.model.eval()
print()

loading alias openai_msmarco__msl128__100epoch__correct from /home/jxm3/research/retrieval/inversion/saves/d6ec9d5838a4ad3daeba636e5378a8a0...
loading alias openai_msmarco__msl128__100epoch from /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f...


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already u

================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
running openai on text_list of length 1, first element 'Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.'
	Decoded output shape ->  torch.Size([1, 69])
	Decoded output -> And the trolls, twilight, was smooty, And swiggy, smeared, And wispy, wisping, wily, wiz, wizz, wizzy, wizzie, wizy, wit.
================ End trainer sanity check ================
Froze 353779584 params from model type <class 'models.inversion.InversionModel'>


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
running openai on text_list of length 1, first element 'Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.'
running openai on text_list of length 1, first element 'And the trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls,'
running openai on text_list of length 1, first element 'Was brilliant, and witty, And wry, writhing, whirling, wit, twig, thy smoots, smeot, riss in the smolly.'
	Decoded output shape ->  torch.Size([1, 62])
	Decoded output -> Was brilliant, and witty, And wry, writhing, whirling, wit, twig, thy smoots, smeot, riss in the smolly.
================ End trainer sanity check ================
Loading hypotheses from path /home/jxm3/research/retrieval/inv

In [3]:
import torch
def bleu(t1: torch.Tensor, t2: torch.Tensor) -> float:
    s1 = corr_trainer.embedder_tokenizer.decode(t1, skip_special_tokens=True)
    s2 = corr_trainer.embedder_tokenizer.decode(t2, skip_special_tokens=True)
    result = corr_trainer.metric_bleu.compute(
        predictions=[s1], references=[s2]
    )
    return result["score"]

In [ ]:
import random

corr_trainer.args.per_device_eval_batch_size = 128
gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}

################################################################################
class EncourageTrueTokensLogitsProcessor(transformers.LogitsProcessor):
    true_input_ids: torch.LongTensor
    def __init__(self, true_input_ids: torch.LongTensor):
        self.true_input_ids = true_input_ids
        # self.gammas = torch.tensor([
        #     random.random() * 10 for _ in range(len(true_input_ids))], 
        #     dtype=torch.float32
        # )
        
    def __call__(
        self, input_ids: torch.LongTensor, next_token_logits: torch.FloatTensor
    ) -> torch.FloatTensor:
        vocab_size = next_token_logits.shape[1]
        
        true_next_tokens = self.true_input_ids[:, input_ids.shape[1]-1]
        fake_logits = torch.zeros_like(next_token_logits, device=next_token_logits.device)
        # todo vectorize
        for i in range(input_ids.shape[0]):
            fake_logits[i, true_next_tokens[i]] += random.random()
        
        # gamma = self.gammas[:, None].to(next_token_logits.device)
        gamma = 8
        logits = (next_token_logits + fake_logits * gamma).log_softmax(1)
        # import pdb; pdb.set_trace()
        
        return logits
################################################################################

eval_dataset = corr_trainer.eval_dataset["msmarco"].select(range(1024))
bleus = []
for batch in corr_trainer.get_eval_dataloader(eval_dataset=eval_dataset):
    true_tokens_logits_processor = EncourageTrueTokensLogitsProcessor(
        true_input_ids=batch["input_ids"],
    )

    gen_kwargs["logits_processor"] = transformers.LogitsProcessorList(
        [
            true_tokens_logits_processor,
        ]
    )
    batch_on_device = { k: v.to(corr_trainer.args.device) for k,v in batch.items() }
    outputs = corr_trainer.generate(
        inputs=batch_on_device,
        generation_kwargs=gen_kwargs,
        num_recursive_steps=1
    )
    bleus.extend([bleu(t1, t2) for t1, t2 in zip(outputs, batch["input_ids"])])

print(sum(bleus) / len(bleus))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# https://seaborn.pydata.org/generated/seaborn.displot.html#seaborn.displot
ax = sns.histplot(bleus, bins=25, color=sns.color_palette()[3])

# ax.set_xlim([.85,1])
# ax.set_ylim([0, 3_000])
ax.set_xlabel("BLEU")
# ax.set_xscale("log")

ax.set_title("Hypothesis BLEU scores")

# save to PDF
# plt.savefig("train_hypothesis_distance_sl128.pdf", format="pdf", bbox_inches="tight") 

In [68]:
(torch.tensor(bleus) >= 90).float().mean()

tensor(0.0283)